### Import Necessary Packages

In [1]:
import boto3 
import gc
import numpy as np
import sagemaker 
from sagemaker import get_execution_role
import os

### Define Bucket and Prefix for Training and Validation data 

In [2]:
bucket = sagemaker.Session().default_bucket()
prefix = 'demand-prediction'

### Configure Necessary Parameters 

In [3]:
algo_name = 'lightgbm'
ecr_repository_name = algo_name 
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
print(account_id)
print(region)

230755935769
us-east-1


### Build and Push Docker Image to ECR 

In [4]:
%%bash -s "$account_id" "$region"
cd light_gbm_sagemaker/script 
./build_and_push.sh $1 $2 lightgbm


Step 1/7 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
2c11b7cecaa5: Pulling fs layer
04637fa56252: Pulling fs layer
d6e6af23a0f3: Pulling fs layer
b4a424de92ad: Pulling fs layer
b4a424de92ad: Waiting
d6e6af23a0f3: Verifying Checksum
d6e6af23a0f3: Download complete
04637fa56252: Verifying Checksum
04637fa56252: Download complete
b4a424de92ad: Verifying Checksum
b4a424de92ad: Download complete
2c11b7cecaa5: Verifying Checksum
2c11b7cecaa5: Download complete
2c11b7cecaa5: Pull complete
04637fa56252: Pull complete
d6e6af23a0f3: Pull complete
b4a424de92ad: Pull complete
Digest: sha256:bb69f1a2b6c840b01eeffef07386e95a74ecac8bfa52d35c32fb526c85e00678
Status: Downloaded newer image for ubuntu:16.04
 ---> dfeff22e96ae
Step 2/7 : RUN apt-get update &&     apt-get install -y --no-install-recommends         ca-certificates         cmake         build-essential         gcc         g++         git &&     rm -rf /var/lib/apt/lists/*
 ---> Running in ef9fd53e27b8
Get:1 http://archive.ubuntu

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Train Model by the Docker Image Just Pushed to ECR

In [ ]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)


In [ ]:
import sagemaker
import time

job_name = "{}-{}".format(algo_name, str(time.time()))
est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix)

est.set_hyperparameters(num_iterations=100)
train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/validation/'.format(bucket, prefix), content_type='text/csv')

est.fit({'train': train_config, 'valid': val_config })
